Welcome to the Rust tutorial for C++ programmers.

Rust helps you to eliminate all memory safety and data race conditions from your C++ programs, while maintaining it's performance (and even make it faster, by removing defensive programming technics and more optimizations as a result of better static knowledge compiler has about the program) and improving it's readability and quality (by modern language constructs, like pattern matching).

This tutorial tries to teach you Rust and convince you that Rust is a great alternative to C++ for your next project.

This is useful for you if:
* You know a fair bit of C++
  * stack and heap
  * C pointers
  * Smart pointers
  * Closures and capture modes
  * Race conditions and Mutex
* You don't think / afraid that Rust is a complex language. Because we will start from most complex aspects of Rust.
* You can learn by reading code. This tutorial has more code than word. The format is such that a topic is raised and then we see some related code about it along with the results. 

## A taste of Rust syntax

Here we have some examples to explain Rust syntax. You can skip this part if you feel confident about guessing meanings of code and jump to the `01-Thread-Safety`

## Variables

In [2]:
let x = 2; // const auto x = 2;
let y: i64 = 5; // const int64_t y = 5;
x + y

7

In [2]:
let x = 2;
x += 3;
x

Error: cannot assign twice to immutable variable `x`

In [3]:
let mut x = 2; // auto x = 2;
x += 3;
x

5

## Control flow

In [4]:
if x < y { // no parenthesis needed, but blocks are mandatory even for single line
    println!("x < y");
} else {
    println!("y < x");
}
println!("after if");

x < y
after if


In [10]:
let mut i = 0;
while i < 5 { // exactly equal to C while
    i += 1;
    println!("{} * {} = {}", i, i, i * i);
}
i

1 * 1 = 1
2 * 2 = 4
3 * 3 = 9
4 * 4 = 16
5 * 5 = 25


5

In [16]:
for i in vec!["foo", "bar", "baz"] { // for (const auto i: std::vector({1, 2, 3}))
    println!("i is {i}"); // single identifiers can be included inside {} in println! and friends
}
for i in 1..=5 { // Rust has builtin range types
    println!("{} * {} = {}", i, i, i * i);
}
// ranges are useful even outside of for loops
// 1..10 is similar to 1..=9
let vec: Vec<i32> = (1..10).collect();
// operator [] is overloaded for ranges, and 3.. is an open ended range.
&vec[3..]

i is foo
i is bar
i is baz
1 * 1 = 1
2 * 2 = 4
3 * 3 = 9
4 * 4 = 16
5 * 5 = 25


[4, 5, 6, 7, 8, 9]

## Scoping

Scoping and shadowing is similar to C:

In [17]:
let x = 1;
{
    let x = "hello";
    println!("{x}");
}
println!("{x}");

hello
1


In Rust, blocks are expressions:

In [19]:
let x = {
    let a = 2;
    let b = 5;
    a * b // note: there is no ; here!
};
x + 3

13

And so ifs:

In [21]:
let x = 5;
let y = 3;
let max = if x > y { x } else { y };
max

5

## Functions

Last expression of a function will be returned, similar to block expressions:

In [25]:
fn euclid_diff(x1: f64, y1: f64, x2: f64, y2: f64) -> f64 {
    let x_diff = x1 - x2;
    let y_diff = y1 - y2;
    (x_diff * x_diff + y_diff * y_diff).sqrt()
}

euclid_diff(0., 0., 3., 3.99)

4.992003605767929

But return is also available:

In [26]:
fn foo(x: i32) -> i32 {
    if x == 0 {
        return -5;
    }
    let y = x + 2;
    y * x
}

foo(0)

-5

Functions without `-> return_type` are void (`()` in Rust):

In [4]:
fn foo(x: i32) {
    if x == 0 {
        return;
    }
    let y = x + 2;
    println!("{y}");
}

foo(12);

14


Adding a semicolon at the end of expression, make it `()`:

In [2]:
fn foo(x: i32) -> i32 {
    if x == 0 {
        return -5;
    }
    let y = x + 2;
    y * x;
}

Error: mismatched types

## Primitives

Numeric primitives:

In [9]:
let a: u8 = 2; // uint8_t
let b: u16 = 0x22; // uint16_t, hex
let c: i64 = -100_000_000_000;
let d = 12_u64;
let e = 1000; // default is i32
let f: f32 = 1.2; // 32 bit ieee floating point, float in C
let g: f64 = std::f64::consts::PI; // 64 bit ieee floating point, double in C
println!("{a} {b} {c} {d} {e} {f} {g}");

2 3000 -100000000000 12 1000 1.2 3.141592653589793


Rust is pedantic on numeric casts, and disallows numeric operations on different types:

In [10]:
let a: u8 = 5;
let b: i16 = 8;
a + b

Error: mismatched types

Error: cannot add `i16` to `u8`

You need to manually cast:

In [14]:
let a: u8 = 5;
let b: i16 = 8;
let c = 10; // will be an i16 automatically
i16::from(a) + b + c

23

String and characters:

In [17]:
// char is a 4 byte value (unlike 1 byte in C) which represents a valid unicode character
let x: char = 'a';
let y = char::from(97);
let z: char = '\u{1F980}';
println!("{x} {y} {z}");

a a 🦀


String literals are utf-8 encoded list of characters, stored in the program's binary. You can have a pointer to them (like C):

In [19]:
let hello_world: &str = "Hello world!";
let xy_crab = "xy🦀";
println!("{}", hello_world.len());
println!("{}", xy_crab.len()); // 2 bytes for x and y, and 4 byte for the crab
println!("{}", xy_crab.chars().count());


12
6
3


## Data types

Rust supports C like structs:

In [22]:
struct Point {
    x: f64,
    y: f64,
}

let p1 = Point { x: 2.3, y: 5.7 };
let p2: Point = p1;
println!("{}, {}", p2.x, p2.y);

2.3, 5.7


Rust also supports tuple struct, which is not available in c/C++:

In [24]:
struct TuplePoint(f64, f64);

let p = TuplePoint(1.0, 1000.55);
println!("{}, {}", p.0, p.1);

1, 1000.55


Rust also has unnamed tuples, which plays rule of `pair` or `tuple` in C++:

In [26]:
let tuple: (u8, &str, char) = (12, "hello", 'x');
let tuple2 = (tuple.2, tuple.1, tuple.0);
tuple2

('x', "hello", 12)

Rust also has arrays:

In [29]:
let mut x: [u8; 3] = [1, 2, 5];
let y = [10, 20];
x[1] = y[0];
x

[1, 10, 5]

## Closure

Rust support closures (functions that can capture environment) like C++. By default, it captures everything by reference (similar to `[&]` in C++):

In [32]:
{ // blocks are due jupyter (evcxr) limitations
    let mut x = 5;
    let mut counter = || {
        x += 1;
        x
    };
    println!("{}", counter());
    println!("{}", counter());
    println!("{}", counter());
    println!("{}", counter());
    println!("{}", x);
};

6
7
8
9
9


But `move` closures will capture everything by value. Since `=` operator means move in Rust, this mode is roughly similar to `[=]` in C++.

In [33]:
{ // blocks are due jupyter (evcxr) limitations
    let mut x = 5;
    let mut counter = move || {
        x += 1;
        x
    };
    println!("{}", counter());
    println!("{}", counter());
    println!("{}", counter());
    println!("{}", counter());
    println!("{}", x); // different!!!
};

6
7
8
9
5


Rust doesn't have syntax for declaring specific capture policy for each variable, but we can do it via move closures:

In [37]:
{
    let mut x = 5;
    let mut y = 5;
    // capture x by reference, y by value
    let mut closure = {
        let x = &mut x; // move reference of x into closure, so capture x by reference
        move || {
            *x += 1;
            y += 1;
        }
    };
    closure();
    closure();
    println!("{x} {y}");
};

7 5


## Namespaces and project structure

Rust has a thing similar to namespace in C++, called module:

In [42]:
mod foo {
    pub fn foo_member() -> i32 {
        5
    }
}

foo::foo_member()

5

We can use `use` to bring items inside the current scope: 

In [45]:
mod a {
    pub fn a_member1(x: i32) -> i32 {
        x + 2
    }
    pub fn a_member2(x: i32) -> i32 {
        x + 5
    }
}

mod b {
    // super is a special keyword, similar to ../ in a file system
    use super::a::{a_member1, a_member2};

    pub fn b_member(x: i32) -> i32 {
        a_member1(a_member2(x))
    }
}

fn foo(x: i32) -> i32 {
    use b::b_member; // use works inside a function
    use a::*; // anything from a
    a_member1(b_member(x) - 3)
}

foo(5)

11

Rust doesn't have concept of header files, and it doesn't use macros (`#include` and friends) for organizing project in multiple files. A compilation unit in Rust is a `crate`, and crates can depend on each other. Compilation flow is similar to C/C++, compiler will compile each crate separately, and then will link the result with a traditional linker.

Let's start with the standard library, `std`, which compiler will add as a dependency to each crate, unless it explicitly opt out with `#![no_std]`. Each dependency is available by it's name, so we can use it and it's submodules: 

In [5]:
// using with use
use std::io::prelude::*;
use std::str::from_utf8;

// or even directly without use
let mut connection = std::net::TcpStream::connect("www.google.com:80").unwrap();
let mut read_buffer = [0; 1024];
connection.write("GET / HTTP/1.1\r\n\r\n".as_bytes()).unwrap();
connection.read(&mut read_buffer).unwrap();
from_utf8(&read_buffer)

Ok("HTTP/1.1 200 OK\r\nDate: Wed, 24 Aug 2022 17:39:51 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nP3P: CP=\"This is not a P3P policy! See g.co/p3phelp for more info.\"\r\nServer: gws\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\nSet-Cookie: 1P_JAR=2022-08-24-17; expires=Fri, 23-Sep-2022 17:39:51 GMT; path=/; domain=.google.com; Secure\r\nSet-Cookie: AEC=AakniGPpyKYdnZXB8IWkCmqNu5L1LjmDvhjRBag4OrGAxa_fipPpbse1uo8; expires=Mon, 20-Feb-2023 17:39:51 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax\r\nSet-Cookie: NID=511=tEMZKlL2Z0xGWtQDMahbpb1QsWzcly-aSazHYhWgh2snF57QW7pRHNkV2a2EOXhuwjqCjnh2P5tJgUS4D7pXJjJpOZX_c8kLuZpC9tJG9jgcF9eaAp2_oJDZBL5H7cW8TOxELT9I9C6p6dPTRmH7BRahIEg8G84dZzdc4Kq_4s4; expires=Thu, 23-Feb-2023 17:39:51 GMT; path=/; domain=.google.com; HttpOnly\r\nAccept-Ranges: none\r\nVary: Accept-Encoding\r\nTransfer-Encoding: chunked\r\n\r\n572a\r\n<!doctype html><html itemscope=\"\" itemtype=\

Standard library provides foundations, but it doesn't have everything for every use case, so we need third party libraries. Rust has a package registry, called `crates.io`, which works with the Rust's package management system, `cargo` and provides an experience similar to `pip` and `npm` and similar.

Normally we would add package name and version to the `Cargo.toml` file. Here in jupyter, we can use `:dep`. Let's add a dependency for fetching http requests: